# Demo - Performance Comparison (CIFAR10)

## 1. Load CIAFR10

In [1]:
# https://github.com/RobustBench/robustbench
from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy

x_test, y_test = load_cifar10(n_examples=50)

Files already downloaded and verified


## 2. Standard Accuracy

In [2]:
model_list = ['Wong2020Fast', 'Rice2020Overfitting', 'Carmon2019Unlabeled']
for model_name in model_list:
    model = load_model(model_name, norm='Linf').cuda()
    acc = clean_accuracy(model, x_test.cuda(), y_test.cuda())
    print('Model: {}'.format(model_name))
    print('- Standard Acc: {}'.format(acc))

Model: Wong2020Fast
- Standard Acc: 0.92
Model: Rice2020Overfitting
- Standard Acc: 0.86
Model: Carmon2019Unlabeled
- Standard Acc: 0.92


## 3. Torchattacks, Foolbox and ART

In [3]:
import datetime
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

# https://github.com/Harry24k/adversarial-attacks-pytorch
import torchattacks
print("torchattacks %s"%(torchattacks.__version__))

# https://github.com/bethgelab/foolbox
import foolbox as fb
print("foolbox %s"%(fb.__version__))

# https://github.com/IBM/adversarial-robustness-toolbox
import art
import art.attacks.evasion as evasion
from art.classifiers import PyTorchClassifier
print("art %s"%(art.__version__))

torchattacks 2.12.1
foolbox 3.0.0
art 1.2.0


## 3.1. Linf

### FGSM

In [4]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').cuda()
    
    print("- Torchattacks")
    atk = torchattacks.FGSM(model, eps=8/255)
    start = datetime.datetime.now()
    adv_test = atk(x_test, y_test)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_test, y_test)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.LinfFastGradientAttack(random_start=False)
    start = datetime.datetime.now()
    _, adv_test, _ = atk(fmodel, x_test.to('cuda:0'), y_test.to('cuda:0'), epsilons=8/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_test, y_test)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.FastGradientMethod(norm=np.inf, batch_size=50,
                                     classifier=classifier, eps=8/255)
    start = datetime.datetime.now()
    advs = atk.generate(x_test, y_test)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_test, y_test)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.48 (31 ms)
- Foolbox
- Robust Acc: 0.48 (24 ms)
- ART
- Robust Acc: 0.48 (71 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.62 (87 ms)
- Foolbox
- Robust Acc: 0.62 (55 ms)
- ART
- Robust Acc: 0.62 (728 ms)

Model: Carmon2019Unlabeled
- Torchattacks
- Robust Acc: 0.68 (23 ms)
- Foolbox
- Robust Acc: 0.68 (33 ms)
- ART
- Robust Acc: 0.68 (217 ms)



### BIM

In [5]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').cuda()
    
    print("- Torchattacks")
    atk = torchattacks.BIM(model, eps=8/255, alpha=2/255, steps=5)
    start = datetime.datetime.now()
    adv_test = atk(x_test, y_test)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_test, y_test)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.LinfBasicIterativeAttack(abs_stepsize=2/255, steps=5, random_start=False)
    start = datetime.datetime.now()
    _, adv_test, _ = atk(fmodel, x_test.to('cuda'), y_test.to('cuda'), epsilons=8/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_test, y_test)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.BasicIterativeMethod(batch_size=50,
                                       classifier=classifier, eps=8/255,
                                       eps_step=2/255, max_iter=5)
    start = datetime.datetime.now()
    advs = atk.generate(x_test, y_test)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_test, y_test)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.46 (63 ms)
- Foolbox
- Robust Acc: 0.46 (73 ms)
- ART
- Robust Acc: 0.46 (249 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.58 (648 ms)
- Foolbox
- Robust Acc: 0.58 (1152 ms)
- ART
- Robust Acc: 0.58 (2486 ms)

Model: Carmon2019Unlabeled
- Torchattacks
- Robust Acc: 0.64 (127 ms)
- Foolbox
- Robust Acc: 0.64 (263 ms)
- ART
- Robust Acc: 0.64 (746 ms)



### PGD

In [6]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').cuda()
    
    print("- Torchattacks")
    atk = torchattacks.PGD(model, eps=8/255, alpha=2/255, steps=5, random_start=False)
    start = datetime.datetime.now()
    adv_test = atk(x_test, y_test)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_test, y_test)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.LinfPGD(abs_stepsize=2/255, steps=5, random_start=False)
    start = datetime.datetime.now()
    _, adv_test, _ = atk(fmodel, x_test.to('cuda'), y_test.to('cuda'), epsilons=8/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_test, y_test)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.ProjectedGradientDescent(batch_size=50, num_random_init=0,
                                           classifier=classifier, eps=8/255,
                                           eps_step=2/255, max_iter=5)
    start = datetime.datetime.now()
    advs = atk.generate(x_test, y_test)
    end = datetime.datetime.now()
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.46 (55 ms)
- Foolbox
- Robust Acc: 0.46 (71 ms)
- ART
- Robust Acc: 0.46 (239 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.58 (583 ms)
- Foolbox
- Robust Acc: 0.58 (1143 ms)
- ART
- Robust Acc: 0.58 (2485 ms)

Model: Carmon2019Unlabeled
- Torchattacks
- Robust Acc: 0.64 (87 ms)
- Foolbox
- Robust Acc: 0.64 (255 ms)
- ART
- Robust Acc: 0.64 (745 ms)



## 3.2. L2

### CW

In [7]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').cuda()
    
    print("- Torchattacks")
    atk = torchattacks.CW(model, c=1, kappa=0, steps=100, lr=0.01)
    start = datetime.datetime.now()
    adv_test = atk(x_test, y_test)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_test, y_test)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc,nn.MSELoss()(x_test.cuda(), adv_test),
                                                         int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.L2CarliniWagnerAttack(binary_search_steps=1, initial_const=1,
                                           confidence=0, steps=100, stepsize=0.01)
    start = datetime.datetime.now()
    _, adv_test, _ = atk(fmodel, x_test.to('cuda'), y_test.to('cuda'), epsilons=1)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_test, y_test)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc,nn.MSELoss()(x_test.cuda(), adv_test),
                                                         int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.CarliniL2Method(batch_size=50, classifier=classifier, 
                                  binary_search_steps=1, initial_const=1,
                                  confidence=0, max_iter=100,
                                  learning_rate=0.01)
    start = datetime.datetime.now()
    advs = atk.generate(x_test, y_test)
    end = datetime.datetime.now()
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc,nn.MSELoss()(x_test.cuda(), adv_test),
                                                         int((end-start).total_seconds()*1000)))
    
    print()

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.14 / L2: 0.00016 (4367 ms)
- Foolbox
- Robust Acc: 0.32 / L2: 0.00016 (4530 ms)
- ART
- Robust Acc: 0.32 / L2: 0.00016 (71613 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.22 / L2: 0.00013 (43680 ms)
- Foolbox
- Robust Acc: 0.34 / L2: 0.00017 (45273 ms)
- ART
- Robust Acc: 0.34 / L2: 0.00017 (691977 ms)

Model: Carmon2019Unlabeled
- Torchattacks
- Robust Acc: 0.26 / L2: 8.5e-05 (13032 ms)
- Foolbox
- Robust Acc: 0.32 / L2: 0.00016 (13314 ms)
- ART
- Robust Acc: 0.32 / L2: 0.00016 (206250 ms)

